In [ ]:
#| echo: false
import logging
import warnings
warnings.simplefilter('ignore')
logging.getLogger('statsforecast').setLevel(logging.ERROR)

# Generating Sample Forecast Trajectories

> Learn how to generate sample forecast paths for probabilistic forecasting and simulation

## Introduction

While prediction intervals provide a useful summary of forecast uncertainty, **sample trajectories** provide a richer representation by generating many possible future scenarios.

This is especially useful for:

- **Monte Carlo simulation**: Simulating many possible futures
- **Coherent probabilistic forecasts**: Samples that respect temporal dependencies
- **Risk analysis**: Understanding the range of possible outcomes

The `generate()` method provides this capability for `AutoARIMA`, `AutoETS`, and `AutoCES` models.

## Install Libraries

In [ ]:
# !pip install statsforecast

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsforecast.models import AutoARIMA, AutoETS, AutoCES

## Load Data

In [ ]:
# Generate sample data with trend and seasonality
np.random.seed(42)
n = 120  # 10 years of monthly data
t = np.arange(n)
trend = 0.1 * t
seasonal = 10 * np.sin(2 * np.pi * t / 12)
noise = np.random.normal(0, 2, n)
y = 100 + trend + seasonal + noise

dates = pd.date_range('2015-01-01', periods=n, freq='M')
df = pd.DataFrame({'date': dates, 'value': y})
df.set_index('date', inplace=True)

plt.figure(figsize=(12, 4))
plt.plot(df.index, df['value'])
plt.title('Monthly Time Series')
plt.show()

## Generate Sample Trajectories with AutoARIMA

In [ ]:
model_arima = AutoARIMA(season_length=12)
model_arima.fit(y)

h = 24
n_samples = 500
samples_arima = model_arima.generate(h=h, n_samples=n_samples, random_state=42)
print(f"Sample trajectories shape: {samples_arima.shape}")

In [ ]:
future_dates = pd.date_range(df.index[-1] + pd.DateOffset(months=1), periods=h, freq='M')

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df.index[-36:], df['value'][-36:], 'b-', linewidth=2, label='Historical')
for i in range(100):
    ax.plot(future_dates, samples_arima[i], 'gray', alpha=0.1, linewidth=0.5)
mean_forecast = samples_arima.mean(axis=0)
ax.plot(future_dates, mean_forecast, 'r-', linewidth=2, label='Mean Forecast')
lower_95 = np.percentile(samples_arima, 2.5, axis=0)
upper_95 = np.percentile(samples_arima, 97.5, axis=0)
ax.fill_between(future_dates, lower_95, upper_95, alpha=0.3, color='red', label='95% PI')
ax.set_title('AutoARIMA: Sample Forecast Trajectories')
ax.legend()
plt.show()

## Generate with AutoETS

In [ ]:
model_ets = AutoETS(season_length=12)
model_ets.fit(y)
samples_ets = model_ets.generate(h=h, n_samples=n_samples, random_state=42)
print(f"AutoETS shape: {samples_ets.shape}")

## Generate with AutoCES

In [ ]:
model_ces = AutoCES(season_length=12)
model_ces.fit(y)
samples_ces = model_ces.generate(h=h, n_samples=n_samples, random_state=42)
print(f"AutoCES shape: {samples_ces.shape}")

## Reproducibility

In [ ]:
samples1 = model_arima.generate(h=12, n_samples=10, random_state=123)
samples2 = model_arima.generate(h=12, n_samples=10, random_state=123)
print("Samples identical:", np.allclose(samples1, samples2))

## Summary

| Model | Method | Description |
|-------|--------|-------------|
| `AutoARIMA` | `.generate(h, n_samples, bootstrap, random_state)` | Kalman state-space simulation |
| `AutoETS` | `.generate(h, n_samples, bootstrap, random_state)` | State-space simulation |
| `AutoCES` | `.generate(h, n_samples, bootstrap, random_state)` | Complex state simulation |